In [1]:
from datasets import load_dataset
dataset = load_dataset("bentrevett/multi30k")
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/29000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1014 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [3]:
dataset['train'][7]

{'en': 'A trendy girl talking on her cellphone while gliding slowly down the street.',
 'de': 'Ein schickes Mädchen spricht mit dem Handy während sie langsam die Straße entlangschwebt.'}

In [ ]:
import spacy

In [ ]:
!python -m spacy download de_core_news_sm
en_nlp=spacy.load('en_core_web_sm')
gm_nlp=spacy.load('de_core_news_sm')

In [ ]:
li=[token.text.lower() for token in en_nlp.tokenizer(dataset['train'][1]['en'])]

In [ ]:
li

In [ ]:
def tokenize(dict_data):
  en_tokens=[token.text.lower() for token in en_nlp.tokenizer(dict_data['en'])]
  gm_tokens=[token.text.lower() for token in en_nlp.tokenizer(dict_data['de'])]
  en_tokens=['<SOS>']+en_tokens+["<EOS>"]
  gm_tokens=['<SOS>']+gm_tokens+["<EOS>"]
  return {"en_tokens": en_tokens, "gm_tokens": gm_tokens}


In [ ]:
train_data=dataset['train'].map(tokenize)
validation_data=dataset['validation'].map(tokenize)
test_data=dataset['test'].map(tokenize)



1. \<sos> :  Start of Sequence. It is used when we want our decoder to predict the starting word of the sequence. We use this token as input to the decoder.


2. \<eos> : End of Sequence. It is used at the end of the sequence. When the decoder predicts this token, we stop the sentence there.


3. \<unk> : Unknown Token. Since our vocabulary is limited to the dataset, we may encounter a situation where we are given a word that we cannot map to a number because it is not in our vocabulary. This is called the out-of-vocabulary (OOV) problem. To prevent this situation, we mark each unknown word with the \<unk> token


4. \<pad> : Padding Token. Padding tokens are used to ensure efficient computation. We know that not all sequences are of the same length — i.e., each sentence does not have the same number of words. However, during computation, we need to have sequences of equal length. The padding token helps make each tokenized sentence the same length by adding extra tokens where needed



In [ ]:
from collections import Counter
import json
from itertools import islice
from typing import Iterable, List

In [ ]:


# SPECIALS (choose your IDs)
PAD = "<PAD>"
UNK = "<UNK>"
SOS = "<SOS>"
EOS = "<EOS>"
SPECIAL_TOKENS = [PAD, UNK, SOS, EOS]

def count_tokens(token_stream: Iterable[List[str]],counter:Counter, max_lines=None, progress=None,token_type=None):
    """Count tokens from an iterator of token lists (streamable)."""

    for i, tokens in enumerate(token_stream):
        counter.update(tokens[token_type])
        if max_lines and i+1 >= max_lines:
            break
        if progress and (i+1) % progress == 0:
            print(f"counted {i+1} lines")

def build_vocab_from_counter(counter: Counter,
                             specials=SPECIAL_TOKENS,
                             min_freq=2,
                             max_size=None):
    # Start by reserving indices for specials
    word2idx = {}
    idx = 0
    for s in specials:
        word2idx[s] = idx
        idx += 1

    # Filter by min_freq, sort by freq desc then token asc for ties
    items = [(w, c) for w, c in counter.items() if c >= min_freq and w not in specials]
    items.sort(key=lambda wc: (-wc[1], wc[0]))

    # If max_size given, keep top (max_size - len(specials))
    if max_size:
        items = items[: max_size - len(specials)]

    for w, _ in items:
        word2idx[w] = idx
        idx += 1

    idx2word = {i:w for w,i in word2idx.items()}
    return word2idx, idx2word




In [ ]:
en_counter=Counter()
count_tokens(train_data,en_counter,token_type='en_tokens')

In [ ]:
gm_counter=Counter()
count_tokens(train_data,gm_counter,token_type='gm_tokens')

In [ ]:
len(en_counter)

In [ ]:
len(gm_counter)

In [ ]:
en_vocab=build_vocab_from_counter(en_counter)

In [ ]:
gm_vocab=build_vocab_from_counter(gm_counter)

In [ ]:
train_data[1]['gm_tokens']

In [43]:
def numericalize_example(example,
                         en_word2idx,
                         gm_word2idx,
                         add_sos_eos=False,
                         sos="<SOS>",
                         eos="<EOS>",
                         unk="<UNK>"):
    # get unk indices (fall back to 1 or 0 if user forgot)
    en_unk = en_word2idx.get(unk)
    gm_unk = gm_word2idx.get(unk)

    if en_unk is None or gm_unk is None:
        raise ValueError("Your vocab must contain the <unk> token.")

    # local speedups
    e2i = en_word2idx
    g2i = gm_word2idx

    # helper to map a list of tokens to ids with unk fallback
    def map_tokens_to_ids(tokens, mapping, unk_idx):
        out = []
        for tok in tokens:
            if not isinstance(tok, str):
                tok = str(tok)
            out.append(mapping.get(tok, unk_idx))
        return out

    en_ids = map_tokens_to_ids(example["en_tokens"], e2i, en_unk)
    gm_ids = map_tokens_to_ids(example["gm_tokens"], g2i, gm_unk)

    if add_sos_eos:
        sos_en = e2i.get(sos)
        eos_en = e2i.get(eos)
        sos_gm = g2i.get(sos)
        eos_gm = g2i.get(eos)
        if None in (sos_en, eos_en, sos_gm, eos_gm):
            raise ValueError("Make sure <sos>/<eos> special tokens exist in both vocabs.")
        en_ids = [sos_en] + en_ids + [eos_en]
        gm_ids = [sos_gm] + gm_ids + [eos_gm]

    return {"en_ids": en_ids, "gm_ids": gm_ids}

train_data = train_data.map(lambda ex:numericalize_example(ex,en_vocab[0],gm_vocab[0]))
valid_data = validation_data.map(lambda ex:numericalize_example(ex,en_vocab[0],gm_vocab[0]))
test_data = test_data.map(lambda ex:numericalize_example(ex,en_vocab[0],gm_vocab[0]))

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [44]:
data_type = "torch"
format_columns = ["en_ids", "gm_ids"]

train_data = train_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [21]:
train_data[1]

{'en_ids': tensor([   2,  113,   30,    6,  325,  279,   17, 1201,    4,  715, 4318, 2878,
            5,    3]),
 'gm_ids': tensor([   2,   78,   32,   11,  844, 2097,    5,    1,    4,    3]),
 'en': 'Several men in hard hats are operating a giant pulley system.',
 'de': 'Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.',
 'en_tokens': ['<SOS>',
  'several',
  'men',
  'in',
  'hard',
  'hats',
  'are',
  'operating',
  'a',
  'giant',
  'pulley',
  'system',
  '.',
  '<EOS>'],
 'gm_tokens': ['<SOS>',
  'mehrere',
  'männer',
  'mit',
  'schutzhelmen',
  'bedienen',
  'ein',
  'antriebsradsystem',
  '.',
  '<EOS>']}

In [22]:
pad_index=en_vocab[0].get("<PAD>")

In [45]:
def collate_fn(batch,pad_index=pad_index):
    en_ids=[i['en_ids'] for i in batch]
    gm_ids=[i['gm_ids'] for i in batch]

    en_ids=torch.nn.utils.rnn.pad_sequence(en_ids,padding_value=pad_index)
    gm_ids=torch.nn.utils.rnn.pad_sequence(gm_ids,padding_value=pad_index)
    return {"en_ids": en_ids, "gm_ids": gm_ids}
batch_size=128
train_dataloader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn,shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=batch_size, collate_fn=collate_fn)



In [46]:
class Encoder(nn.Module):
  def __init__(self,input_dim,embedding_dim,hidden_dim,num_layer,dropout):
    super().__init__()
    self.embedding=nn.Embedding(input_dim,embedding_dim)
    self.rnn=nn.LSTM(embedding_dim,hidden_dim,num_layer,dropout=dropout)

  def forward(self,src):
    # print("Max src ID in batch:", src.max().item(), "Embedding size:", self.embedding.num_embeddings)

    embedded=self.embedding(src) #src-->(sentence_len,batch_size)
    #embedded-->(sentence_len,batch_size,embedding_dim)
    # print("Before LSTM:enc", src.shape)
    output,(h_n,c_n)=self.rnn(embedded)
    #output-->(sentence_len,batch_size,hidden_dim)
    #h_n(1,batch_size,hidden_dim)-->(1*num_layer,batch_size,hidden_dim)
    #c_n(1,batch_size,hidden_dim)-->(1*num_layer,batch_size,hidden_dim)
    return output,h_n,c_n



In [47]:
import torch.nn.functional as F

class LuongAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        # The linear layer Wa, which acts as the "adapter"
        self.attn = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, decoder_hidden, encoder_outputs):
        """
        Calculates the attention scores.

        Args:
            decoder_hidden (Tensor): The current hidden state of the decoder.
                                     Shape: [num_layers, batch_size, hidden_dim]
            encoder_outputs (Tensor): All hidden states from the encoder.
                                      Shape: [src_len, batch_size, hidden_dim]

        Returns:
            Tensor: Attention weights. Shape: [batch_size, 1, src_len]
        """

        # We only use the top layer of the decoder's hidden state as our "query" or "lock"
        # Permute to get shape [batch_size, 1, hidden_dim] for bmm
        query = decoder_hidden[-1].unsqueeze(1)

        # 1. Pass the ENCODER outputs (the "keys") through the adapter (Wa).
        # We permute encoder_outputs to be [batch_size, src_len, hidden_dim] for bmm
        keys = self.attn(encoder_outputs.permute(1, 0, 2))

        # 2. Calculate the dot product (scores) between the lock and the adapted keys.
        # torch.bmm performs batch matrix multiplication.
        # [batch_size, 1, hidden_dim] @ [batch_size, hidden_dim, src_len] -> [batch_size, 1, src_len]
        scores = torch.bmm(query, keys.transpose(1, 2))

        # 3. Apply softmax to get a probability distribution (the weights).
        weights = F.softmax(scores, dim=2)

        return weights

In [48]:
# class AttnDecoder(nn.Module):
#   def __init__(self,output_dim,embedding_dim,hidden_dim,num_layers,dropout):
#     super().__init__()
#     self.embedding=nn.Embedding(output_dim,embedding_dim)
#     self.rnn=nn.LSTM(embedding_dim+hidden_dim,hidden_dim,num_layers,dropout=dropout)
#     self.fc=nn.Linear(hidden_dim,1024)
#     self.fc1=nn.Linear(1024,output_dim)
#     self.relu=nn.ReLU()
#     self.luong=LuongAttention(hidden_dim)
#   def forward(self,src,h_n_dec,c_n_dec,encoder_outputs):
#     # print("Max src ID in batch:", src.max().item(), "Embedding size:", self.embedding.num_embeddings)
#     #src-->(batch_size)
#     embedded=self.embedding(src.unsqueeze(0))
#     #embedded (1,batch_size,embedding_dim) -> (1,2,724)
#     # print("Before LSTM:dec", src.shape)


#     attn_weights=self.luong(h_n_dec,encoder_outputs) #shape:(batch_size,1,seq_length)
#     #encoder_outputs:shape(seq_length,batch_size,hidden_dim)
#     context_vector=torch.bmm(attn_weights,encoder_outputs.permute(1,0,2))
#     #context_vector:shape(batch_size,1,hidden_dim)

#     rnn_input=torch.cat((context_vector.permute(1,0,2),embedded),dim=2)
#     out,(h_n,c_n)=self.rnn(rnn_input,(h_n_dec,c_n_dec))
#     # output (sentence_length,batch_size , hidden_dim) -> (1,2,512) for each timestamp(word) we will have a hiddenlayer output
#     # h_n = (1,batch_size,hidden_dim) -> (1*n_layers,2,512) -> (2,2,512) we get hidden layer output for each sentence last timestamp(word)
#     # c_n = (1,batch_size,hidden_dim) -> (1*n_layers,2,512) -> (2,2,512) we get Cell state for each sentence last timestamp(word)
#     prediction=self.relu(self.fc(out))
#     # prediction (batch,out_features) -> (2,1024)
#     prediction=self.fc1(prediction)
#     # prediction (batch,out_features) -> (2,vocab_size)
#     return prediction, h_n, c_n




In [65]:
class AttnDecoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.attention = LuongAttention(hidden_dim)
        self.rnn = nn.LSTM(embedding_dim + hidden_dim, hidden_dim, num_layers, dropout=dropout)

        # FIX 1: The final output layer should combine the hidden state and context vector.
        # Its input size is hidden_dim + hidden_dim.
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)

        # Calculate attention weights and context vector
        attn_weights = self.attention(hidden, encoder_outputs)
        context_vector = torch.bmm(attn_weights, encoder_outputs.permute(1, 0, 2))
        context_vector = context_vector.permute(1, 0, 2)

        # Concatenate embedding and context vector for the RNN input
        rnn_input = torch.cat((embedded, context_vector), dim=2)

        # Pass input and previous states to the RNN
        rnn_output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))

        # FIX 2: Create the final prediction vector by combining the RNN output
        # and the context vector. This is the final step of Luong attention.

        # Concatenate the RNN output and the context vector
        # rnn_output shape: [1, batch_size, hidden_dim]
        # context_vector shape: [1, batch_size, hidden_dim]
        combined_output = torch.cat((rnn_output, context_vector), dim=2)

        # Pass the combined vector through the final linear layer
        prediction = self.fc_out(combined_output.squeeze(0))
        # prediction shape: [batch_size, output_dim]

        return prediction, hidden, cell

In [27]:
import random

In [66]:
class seq2seq(nn.Module):
  def __init__(self, decoder, encoder, device):
     super().__init__()
     self.decoder=decoder
     self.encoder=encoder
     self.device=device

  def forward(self,src,trg):

     encoder_outputs,h_n_enc,c_n_enc=self.encoder(src)
     sentence_len=trg.shape[0]
     batch_size=trg.shape[1]
     target_vocab=len(en_vocab[0])

     output_vector=torch.zeros(sentence_len,batch_size,target_vocab).to(self.device)
     h_n_dec=h_n_enc
     c_n_dec=h_n_enc
     input=trg[0,:]
     teacher_forcing_ratio=0.5
     for i in range(1,sentence_len):
       prediction,h_n_dec,c_n_dec=self.decoder(input,h_n_dec,c_n_dec,encoder_outputs)

       output_vector[i]=prediction.squeeze(0)

       teacher_force = random.random() < teacher_forcing_ratio
       input_for_next_word_pred = prediction.squeeze(0).argmax(1)
      # input_for_next_word_pred (batch_size) -> (2)
       input = trg[i] if teacher_force else input_for_next_word_pred
      # input (batch_size) -> (2)

     return output_vector



In [29]:
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


In [30]:
for batch in train_dataloader:
    print(batch["en_ids"].max().item(), batch["en_ids"].min().item())
    break


5863 0


In [ ]:
print("gm_vocab size:", len(gm_vocab[0]))
print("max gm_id:", max(max(seq) for seq in train_data['gm_ids']))
print("en_vocab size:", len(en_vocab[0]))
print("max en_id:", max(max(seq) for seq in train_data['en_ids']))


In [67]:
input_dims = len(gm_vocab[0])
output_dims = len(en_vocab[0])
embedding_dim = 724
hidden_dim = 512
num_layers = 2
dropout = 0.5
teacher_forcing_ratio = 0.5
device = "cuda" if torch.cuda.is_available() else "cpu"

In [81]:
encoder = Encoder(input_dims,embedding_dim,hidden_dim,num_layers,dropout)
decoder = AttnDecoder(output_dims,embedding_dim,hidden_dim,num_layers,dropout)

model = seq2seq(decoder,encoder,device).to(device)

In [69]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
model.apply(init_weights)

seq2seq(
  (decoder): AttnDecoder(
    (embedding): Embedding(5893, 724)
    (attention): LuongAttention(
      (attn): Linear(in_features=512, out_features=512, bias=True)
    )
    (rnn): LSTM(1236, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=1024, out_features=5893, bias=True)
  )
  (encoder): Encoder(
    (embedding): Embedding(7874, 724)
    (rnn): LSTM(724, 512, num_layers=2, dropout=0.5)
  )
)

In [70]:
total_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"total number of parameters : {total_parameters}")


total number of parameters : 26592209


In [71]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [55]:
import tqdm

In [37]:
for i, batch in enumerate(train_dataloader):
    src = batch["gm_ids"]
    trg = batch["en_ids"]

    max_src_id = src.max().item()
    max_trg_id = trg.max().item()

    if max_src_id >= len(gm_vocab[0]):
        print(f"Batch {i}: src has invalid ID {max_src_id} >= {len(gm_vocab[0])}")
        break

    if max_trg_id >= len(en_vocab[0]):
        print(f"Batch {i}: trg has invalid ID {max_trg_id} >= {len(en_vocab[0])}")
        break


In [ ]:
best_loss = float("inf")
epochs = 25

for i in range(epochs):
  # model training
  model.train()
  total_loss = 0
  for batch in tqdm.tqdm(train_dataloader):
    src = batch["gm_ids"].to(device)
    # src (batch size , sentence_length) ->(2,n) here 2 is number of sentence and batch size represent number of sentence
    trg = batch["en_ids"].to(device)
    # trg (batch size , sentence_length) ->(2,n)
    # print("Max src ID in batch:", src.max().item(), "src embedding size:", input_dims)
    # print("Max trg ID in batch:", trg.max().item(), "trg embedding size:", output_dims)
    output = model(src,trg)
    # output (sentence_length,batch_size,target_vocabulary) -> (n,2,5893)
    output = output[1:].reshape(-1,output.size(-1))
    # output (sentence_length*batch_size,target_vocabulary) -> (2*n,5893)
    trg = trg[1:].reshape(-1)
    # trg (sentence_length*batch_size) -> (2*n)


    optimizer.zero_grad()
    loss = criterion(output,trg)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  print(f" trainig loss for epoch {i+1} : {total_loss/len(train_dataloader)}")

  # model evaluation
  model.eval()
  total_loss = 0
  with torch.no_grad():
    for batch in tqdm.tqdm(valid_dataloader):
      src = batch["gm_ids"].to(device)
      # src (batch size , sentence_length) ->(2,n) here 2 is number of sentence and batch size represent number of sentence
      trg = batch["en_ids"].to(device)
      # trg (batch size , sentence_length) ->(2,n)


      output = model(src,trg)
      output = output[1:].reshape(-1,output.size(-1))
      # output (sentence_length*batch_size,target_vocabulary) -> (2*n,5893)
      trg = trg[1:].reshape(-1)
      # trg (sentence_length*batch_size) -> (2*n)
      loss = criterion(output,trg)
      total_loss += loss.item()
  print(f" validation loss for epoch {i+1} : {total_loss/len(valid_dataloader)}")
  if total_loss < best_loss:
    best_loss = total_loss
    torch.save(model.state_dict(),"best_model.pt")
    print(f"model saved with validation loss {best_loss/len(valid_dataloader)}")

In [57]:
import joblib

In [73]:
joblib.dump(model,'seq2seq_with_attention2.joblib')

['seq2seq_with_attention2.joblib']

In [60]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 51.9 MB/s eta 0:00:00


In [61]:
from torchmetrics.text.bleu import BLEUScore

In [74]:
def map_ids_to_tokens(ids, mapping):
  out = []
  for id in ids:
      if not isinstance(id, (int,float)):
          id = int(id)
      out.append(mapping.get(id,'<UNK>'))
  return out

In [ ]:
def translate(text, model, gm_nlp, gm_vocab, en_vocab, device, seq_length=50):
    model.eval()

    tokens = [token.text.lower() for token in gm_nlp.tokenizer(text)]
    tokens = ["<SOS>"] + tokens + ["<EOS>"]

    unk_idx = gm_vocab[0]["<UNK>"]
    src_indexes = [gm_vocab[0].get(token, unk_idx) for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    with torch.no_grad():
      
        encoder_outputs, hidden, cell = model.encoder(src_tensor)

    trg_indexes = [en_vocab[0]["<SOS>"]]

    for i in range(seq_length):
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            
            prediction, hidden, cell = model.decoder(trg_tensor, hidden, cell, encoder_outputs)

        pred_token = prediction.argmax(1).item()
        trg_indexes.append(pred_token)

        if pred_token == en_vocab[0]["<EOS>"]:
            break

    trg_tokens = map_ids_to_tokens(trg_indexes, en_vocab[1])

    return trg_tokens[1:-1] if len(trg_tokens) > 1 else []


print("Translating test data for BLEU score calculation...")
candidate_corpus_tokens = [translate(example["de"], model, gm_nlp, gm_vocab, en_vocab, device) for example in test_data]
candidate_corpus = [" ".join(tokens) for tokens in candidate_corpus_tokens]

# 2. Prepare reference translations, JOIN them, and ensure correct format
references_corpus_tokens = [[example["en_tokens"][1:-1]] for example in test_data]
references_corpus = [[" ".join(tokens) for tokens in ref] for ref in references_corpus_tokens]
print(references_corpus)
print(candidate_corpus)
# 3. Initialize and calculate the score with the correctly formatted strings
bleu = BLEUScore()
score = bleu(candidate_corpus, references_corpus)

print(f"\nBLEUScore: {score.item()}")

Translating test data for BLEU score calculation...
[['a man in an orange hat starring at something .'], ['a boston terrier is running on lush green grass in front of a white fence .'], ['a girl in karate uniform breaking a stick with a front kick .'], ['five people wearing winter jackets and helmets stand in the snow , with snowmobiles in the background .'], ['people are fixing the roof of a house .'], ['a man in light colored clothing photographs a group of men wearing dark suits and hats standing around a woman dressed in a strapless gown .'], ['a group of people standing in front of an igloo .'], ['a boy in a red uniform is attempting to avoid getting out at home plate , while the catcher in the blue uniform is attempting to catch him .'], ['a guy works on a building .'], ['a man in a vest is sitting in a chair and holding magazines .'], ['a mother and her young song enjoying a beautiful day outside .'], ['men playing volleyball , with one player missing the ball but hands still in

In [ ]:
# Select an example to translate
index = 200
text = test_data["de"][index] # Using test_data is better for a final check



# 1. Tokenize the input text
tokens = [token.text.lower() for token in gm_nlp.tokenizer(text)]
tokens = ["<SOS>"] + tokens + ["<EOS>"]

# 2. Convert tokens to indices using the dictionary (gm_vocab[0])
unk_idx = gm_vocab[0]["<UNK>"]
src_indexes = [gm_vocab[0].get(token, unk_idx) for token in tokens]
src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

output_indexes = []
model.eval()

with torch.no_grad():
    # 3. Unpack all THREE outputs from the encoder
    encoder_outputs, hidden, cell = model.encoder(src_tensor)

    # 4. Get the <SOS> token index from the dictionary
    trg_input = torch.LongTensor([en_vocab[0]["<SOS>"]]).to(device)

    for i in range(50): 
        # 5. Pass encoder_outputs to the decoder for attention
        prediction, hidden, cell = model.decoder(trg_input, hidden, cell, encoder_outputs)

        # Get the predicted word index
        pred_token_idx = prediction.argmax(1).item()

        # 6.Check for <EOS> using the dictionary (en_vocab[0])
        if pred_token_idx == en_vocab[0]["<EOS>"]:
            break

        output_indexes.append(pred_token_idx)
        trg_input = torch.LongTensor([pred_token_idx]).to(device)

# 7. Convert output indices to tokens using the dictionary (en_vocab[1])
output_tokens = [en_vocab[1][idx] for idx in output_indexes]
output = " ".join(output_tokens)

print("Original German Text: ", text)
print("Actual English Translation: ", test_data["en"][index])
print("Predicted English Translation: ", output)

Original German Text:  Ein Mann wirft einen Ast in einen Kanal während zwei Hunde zusehen.
Actual English Translation:  A man is throwing a log into a waterway while two dogs watch.
Predicted English Translation:  a man throwing a tree in a tree while two others look on two two dogs look on two others .
